In [22]:
import pandas as pd
import pulp

# 1. Load and Clean

In [23]:
def clean_number(x):
    """Remove commas and convert to float."""
    if isinstance(x, str):
        return float(x.replace(',', ''))
    return float(x)

# Load CSVs
df_centers = pd.read_csv('centers.csv')
df_focus = pd.read_csv('focus_cities.csv')
df_hubs = pd.read_csv('hubs.csv')
df_cost_centers = pd.read_csv('costs_centers.csv')
df_cost_focus = pd.read_csv('costs_focus_cities.csv')

# Hubs
df_hubs['Capacity'] = df_hubs['Capacity (Monthly Tons)'].apply(clean_number)
hub_cap = dict(zip(df_hubs['Hub City'], df_hubs['Capacity']))
hub_ids = list(hub_cap.keys())

# Focus Cities
df_focus['Capacity'] = df_focus['Capacity (Monthly Tons)'].apply(clean_number)
focus_cap = dict(zip(df_focus['Focus City'], df_focus['Capacity']))
focus_ids = list(focus_cap.keys())

# Centers
df_centers['Demand'] = df_centers['Demand (Monthly Tons)'].apply(clean_number)
center_demand = dict(zip(df_centers['City'], df_centers['Demand']))
center_ids = list(center_demand.keys())

# 2. Build cost dictionaries for valid routes

In [24]:
# Helpers to extract costs. Returns a dictionary {(Source, Dest): Cost}
x_costs = {} # Hub -> Focus
y_costs = {} # Hub -> Center
z_costs = {} # Focus -> Center

# x: Hub -> Focus
for h in hub_ids:
    for f in focus_ids:
        # Find row where Destination is f
        row = df_cost_focus[df_cost_focus['Destination Focus City'] == f]
        if not row.empty:
            cost = row[h].values[0] # Get cost from Hub column
            if pd.notna(cost):
                x_costs[(h, f)] = float(cost)

# y: Hub -> Center
for h in hub_ids:
    for c in center_ids:
        row = df_cost_centers[df_cost_centers['Destination Center City'] == c]
        if not row.empty:
            cost = row[h].values[0]
            if pd.notna(cost):
                y_costs[(h, c)] = float(cost)

# z: Focus -> Center
for f in focus_ids:
    for c in center_ids:
        row = df_cost_centers[df_cost_centers['Destination Center City'] == c]
        if not row.empty:
            cost = row[f].values[0]
            if pd.notna(cost):
                z_costs[(f, c)] = float(cost)

# Define Lists of Valid Routes/keys of the dictionaries
x_routes = list(x_costs.keys())
y_routes = list(y_costs.keys())
z_routes = list(z_costs.keys())

# 3. Optimization Model with PuLP

In [25]:
prob = pulp.LpProblem("Amazon_Network_Optimization", pulp.LpMinimize)

# Variables
x = pulp.LpVariable.dicts("x", x_routes, lowBound=0, cat='Continuous')
y = pulp.LpVariable.dicts("y", y_routes, lowBound=0, cat='Continuous')
z = pulp.LpVariable.dicts("z", z_routes, lowBound=0, cat='Continuous')

# Objective Function: Minimize Total Cost
prob += (
    pulp.lpSum([x[r] * x_costs[r] for r in x_routes]) +
    pulp.lpSum([y[r] * y_costs[r] for r in y_routes]) +
    pulp.lpSum([z[r] * z_costs[r] for r in z_routes])
)

# Constraints

# 1. Hub Capacity
for h in hub_ids:
    prob += (
        pulp.lpSum([x[(h, f)] for f in focus_ids if (h, f) in x_routes]) +
        pulp.lpSum([y[(h, c)] for c in center_ids if (h, c) in y_routes])
    ) <= hub_cap[h], f"Hub_Cap_{h}"

# 2. Focus City Capacity (Input Limit)
for f in focus_ids:
    prob += (
        pulp.lpSum([x[(h, f)] for h in hub_ids if (h, f) in x_routes])
    ) <= focus_cap[f], f"Focus_Cap_{f}"

# 3. Focus City Flow Balance (Input = Output)
for f in focus_ids:
    prob += (
        pulp.lpSum([x[(h, f)] for h in hub_ids if (h, f) in x_routes]) -
        pulp.lpSum([z[(f, c)] for c in center_ids if (f, c) in z_routes])
    ) == 0, f"Flow_Balance_{f}"

# 4. Center Demand
for c in center_ids:
    prob += (
        pulp.lpSum([y[(h, c)] for h in hub_ids if (h, c) in y_routes]) +
        pulp.lpSum([z[(f, c)] for f in focus_ids if (f, c) in z_routes])
    ) == center_demand[c], f"Demand_{c}"

# Solve
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/zoltesh/projects/optimization_problem/.venv/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/cb02a531ef99463b99ebc359476bf4ba-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/cb02a531ef99463b99ebc359476bf4ba-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 78 COLUMNS
At line 655 RHS
At line 729 BOUNDS
At line 730 ENDATA
Problem MODEL has 73 rows, 191 columns and 385 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 47 (-26) rows, 146 (-45) columns and 294 (-91) elements
Perturbing problem by 0.001% of 1.6 - largest nonzero change 9.0370431e-05 ( 0.018965159%) - largest zero change 4.285993e-05
0  Obj 150023.19 Primal inf 81777.101 (44)
45  Obj 199480.91
Optimal - objective value 199476.25
After Postsolve, objective 199476.25, infeasibilities - dual 0 (0), prima

1

# 4. Results

In [26]:
print(f"Status: {pulp.LpStatus[prob.status]}")
print("Total Cost: ${:,.2f}".format(pulp.value(prob.objective)))

print("\n--- NON-ZERO SHIPMENTS ---")
for v in prob.variables():
    if v.varValue > 0:
        print(f"{v.name} = {v.varValue}")

print("\n--- CONSTRAINT CHECKS ---")

# 1. Check Hub Capacities (Hub Output <= Hub Capacity)
print("\n[Check 1] Hub Capacities:")
for h in hub_ids:
    used = sum([x[(h, f)].varValue for f in focus_ids if (h, f) in x_routes]) + \
           sum([y[(h, c)].varValue for c in center_ids if (h, c) in y_routes])
    print(f"  Hub {h}: Used {used:,.0f} / {hub_cap[h]:,.0f} -> {'OK' if used <= hub_cap[h] + 0.1 else 'FAIL'}")

# 2. Check Focus City Flow Balance (In == Out)
print("\n[Check 2] Focus City Flow Balance (In = Out):")
for f in focus_ids:
    val_in = sum([x[(h, f)].varValue for h in hub_ids if (h, f) in x_routes])
    val_out = sum([z[(f, c)].varValue for c in center_ids if (f, c) in z_routes])
    print(f"  Focus {f}: In {val_in:,.0f} / Out {val_out:,.0f} -> {'OK' if abs(val_in - val_out) < 0.1 else 'FAIL'}")

# 3. Check Focus City Capacities (In <= Focus Capacity)
print("\n[Check 3] Focus City Inbound Capacities:")
for f in focus_ids:
    val_in = sum([x[(h, f)].varValue for h in hub_ids if (h, f) in x_routes])
    cap = focus_cap[f]
    print(f"  Focus {f}: Inbound {val_in:,.0f} / Capacity {cap:,.0f} -> {'OK' if val_in <= cap + 0.1 else 'FAIL'}")

# 4. Check Center Demand (Received == Required)
print("\n[Check 4] Center Demand Fulfillment:")
mismatches = 0
for c in center_ids:
    # Sum of direct shipments from Hubs (y) + shipments from Focus Cities (z)
    received = sum([y[(h, c)].varValue for h in hub_ids if (h, c) in y_routes]) + \
               sum([z[(f, c)].varValue for f in focus_ids if (f, c) in z_routes])
    required = center_demand[c]
    
    # Check if satisfied (allowing for tiny floating point differences)
    if abs(received - required) > 0.1:
        print(f"  FAIL: Center {c} received {received:,.0f} but needed {required:,.0f}")
        mismatches += 1
    
# Print constraint check results
if mismatches == 0:
    print(f"  SUCCESS: All {len(center_ids)} centers received exactly their required demand.")
else:
    print(f"  WARNING: {mismatches} centers did not receive correct demand.")

Status: Optimal
Total Cost: $199,476.25

--- NON-ZERO SHIPMENTS ---
x_('Cincinnati',_'Leipzig') = 43470.0
y_('Alliance',_'Albuquerque') = 450.0
y_('Alliance',_'Allentown') = 420.0
y_('Alliance',_'Anchorage') = 175.0
y_('Alliance',_'Atlanta') = 3000.0
y_('Alliance',_'Austin') = 975.0
y_('Alliance',_'Baltimore') = 1300.0
y_('Alliance',_'Chicago_Rockford') = 172.0
y_('Alliance',_'Denver') = 1500.0
y_('Alliance',_'Des_Moines') = 300.0
y_('Alliance',_'Fairbanks') = 38.0
y_('Alliance',_'Fort_Wayne') = 200.0
y_('Alliance',_'Hartford') = 540.0
y_('Alliance',_'Honolulu') = 500.0
y_('Alliance',_'Houston') = 3300.0
y_('Alliance',_'Kahului_Maui') = 16.0
y_('Alliance',_'Kansas_City') = 975.0
y_('Alliance',_'Kona') = 63.0
y_('Alliance',_'Lakeland') = 185.0
y_('Alliance',_'Manchester') = 100.0
y_('Alliance',_'Miami') = 3400.0
y_('Alliance',_'Minneapolis') = 1700.0
y_('Alliance',_'Nashville') = 650.0
y_('Alliance',_'New_York') = 11200.0
y_('Alliance',_'Phoenix') = 2400.0
y_('Alliance',_'Pittsburgh') =

In [ ]:
# 5. INTERNAL MATH VALIDATION (SPOT CHECK)
print("\n--- INTERNAL MATH VALIDATION ---")
# Find a 'y' route with volume to spot check
for route in y_routes:
    if y[route].varValue > 10000: # A major route
        vol = y[route].varValue
        cost = y_costs[route] # This pulls the actual cost used by the model
        total = vol * cost
        print(f"Spot Check Route: {route}")
        print(f"  Model Volume:   {vol:,.0f}")
        print(f"  Model Cost:     ${cost:.2f} (from input dictionary)")
        print(f"  Calculated cost contribution: ${total:,.2f}")
        break


--- INTERNAL MATH VALIDATION ---
Spot Check Route: ('Alliance', 'New York')
  Model Volume:   11,200
  Model Cost:     $0.50 (from input dictionary)
  Calculated cost contribution: $5,600.00
